# LionAGI - Introduction 2

Nested Data Structure is common for API calls, but they are annoying to handle. `LionAGI` has some powerful functions to handle nested data structures.

In [1]:
import lionagi as li

In [2]:
# let us define a couple nested structure

nested_list = [0, [1, 2, [3, 4]]]

nested_dict = {
    'level1': {
        'level2': {
            'level3': 'some_value'
        }
    }
}

## 1. Nested Structure Operations
- nset
- nget
- is_structure_homogenous

`nset` & `nget`

In [3]:
# suppose you have a nested dictionary as above, how would you access the value?
# typically you can do the following, 

value1 = nested_dict['level1']['level2']['level3'] 

but this can be tiresome, especially after the keys get long and with confusing names 

In [4]:
# nget allows you to extract the value in the nested structure using a list

keys = ['level1', 'level2', 'level3']

value2 = li.nget(nested_dict, keys)

In [5]:
value1 == value2

True

similarly you can use `nset` to set a value in a nested structure

In [6]:
li.nset(nested_dict, keys, 'new_value')
print('Nested dictionary after setting value:')

print(li.to_readable_dict(nested_dict))

Nested dictionary after setting value:
{
    "level1": {
        "level2": {
            "level3": "new_value"
        }
    }
}


`nset` and `nget` work for both nested lists and nested dictionary

In [7]:
indices = [1, 2, 1]
li.nget(nested_list, indices)

4

In [8]:
# Setting a new value at a specific index in the nested list
li.nset(nested_list, indices, 'new_value')
print('Nested list after setting value:', nested_list)

Nested list after setting value: [0, [1, 2, [3, 'new_value']]]


if you try to get a value of an non-existent path, default value is `None`

In [9]:
nested_dict = {
    'level1': {
        'level2': {
            'level3': 'value'
        }
    }
}

# Attempting to get a value at a non-existent path, which returns None
value = li.nget(nested_dict, ['level1', 'level2', 'non_existent_key'])
print('Value from non-existent key (should be None):', value)

Value from non-existent key (should be None): None


One common issue with reading json is that sometimes they can contain lists, 

which makes indexing very confusing, and we might need different handling for pure dictioanry or a mixed type, 

we can use `is_structure_homogenous` to check, whether a given structure is purely nested dictionary / nested lists or not

In [10]:
nested_dict = {
    'a': {'b': {'c': 1}},
    'd': {'e': {'f': 2}}
}

# Check if the nested structure is homogeneous
is_homogeneous, structure_type = li.is_structure_homogeneous(nested_dict, return_structure_type=True)
print('The nested dictionary is homogeneous:', is_homogeneous, '\nType:', structure_type)

The nested dictionary is homogeneous: True 
Type: <class 'dict'>


In [11]:
nested_dict = {
    'a': [1, 2, {'b': 3}],
    'c': {'d': 4}
}

is_homogeneous, structure_type = li.is_structure_homogeneous(nested_dict, return_structure_type=True)
print('The nested structure is homogeneous:', is_homogeneous, '\nType:', structure_type)

The nested structure is homogeneous: False 
Type: None


## 2. Flattening and Unflattening a complex nested data structure
- to_list
- flatten
- unflatten

In [12]:
# we can use to_list to flatten a list of nested lists, you can also use it to dropna

nested_list = [1, [2, [3, None, 4]], [5, 6], None]

print(li.to_list(nested_list, flatten=True, dropna=True))

[1, 2, 3, 4, 5, 6]


we can also flatten a dictionary of complex nested types

In [13]:
nested = {
    'a': [1, 2, {'b': 3}],
    'c': {'d': 4, 
          'e': {'f': [5,6,7]}}
}

print(li.to_readable_dict(nested))

{
    "a": [
        1,
        2,
        {
            "b": 3
        }
    ],
    "c": {
        "d": 4,
        "e": {
            "f": [
                5,
                6,
                7
            ]
        }
    }
}


In [14]:
# now we can flatten it to a 1-d dictionary
flattened = li.flatten(nested)

print(li.to_readable_dict(flattened))

{
    "a_0": 1,
    "a_1": 2,
    "a_2_b": 3,
    "c_d": 4,
    "c_e_f_0": 5,
    "c_e_f_1": 6,
    "c_e_f_2": 7
}


In [15]:
# Flatten the nested dictionary with a max depth of 1
flat_dict = li.flatten(nested, max_depth=1)
print('Flattened nested dictionary with max depth:')
print(li.to_readable_dict(flat_dict))

Flattened nested dictionary with max depth:
{
    "a_0": 1,
    "a_1": 2,
    "a_2": {
        "b": 3
    },
    "c_d": 4,
    "c_e": {
        "f": [
            5,
            6,
            7
        ]
    }
}


if you just want the keys as path, you can use `get_flattened_keys`

In [16]:
keys = li.get_flattened_keys(nested)

print(keys)
print(f"\nNumber of unique key-value pair: {len(keys)}")

['a_0', 'a_1', 'a_2_b', 'c_d', 'c_e_f_0', 'c_e_f_1', 'c_e_f_2']

Number of unique key-value pair: 7


Let's say you have done some operations on the flattened dictionary, and you would like to fold them into an organized nested structure, 

you can use `unflatten`

In [17]:
unflattened = li.unflatten(flattened)

print(unflattened)
print(unflattened==nested)

{'a': [1, 2, {'b': 3}], 'c': {'d': 4, 'e': {'f': [5, 6, 7]}}}
True


## 3. ninsert, nfilter and nmerge

you can insert value into a nested dictionary according to the path, if path doesn't exist, `ninsert` will create it

In [18]:
# create a nested structure by inserting value, key will be created as a nested manner
nested_dict = {}
li.ninsert(nested_dict, ['a', 'b', 'c'], value=1)
li.ninsert(nested_dict, ['a', 'b', 'd'], value=2)
li.ninsert(nested_dict, ['a', 'e'], value=3)
li.ninsert(nested_dict, ['f'], value=4)
print(li.to_readable_dict(nested_dict))

{
    "a": {
        "b": {
            "c": 1,
            "d": 2
        },
        "e": 3
    },
    "f": 4
}


`nfilter` can be used to filter out elements of a nested structure

In [19]:
nested_dict1 = {
    "data": {'temperature': 22, 'humidity': 80, 'pressure': 1012}, 
    "threshold": {'temperature': 20, 'humidity': 85,'pressure': 1000}
}

# finding value larger than threshold
def condition_for_dict1(item):
    key, value = item
    return value > nested_dict1['threshold'][key]

filtered_data1 = li.nfilter(nested_dict1['data'], condition_for_dict1)
print('Filtered nested dictionary:', filtered_data1)

Filtered nested dictionary: {'temperature': 22, 'pressure': 1012}


In [20]:
list_of_dicts = [
    {'name': 'Alice', 'age': 30},
    {'name': 'Bob', 'age': 25},
    {'name': 'Charlie', 'age': 35}
]

# finding people with age larger than 30
def condition_for_list_of_dicts(item):
    return item.get('age', 0) > 30

filtered_list_of_dicts = li.nfilter(list_of_dicts, condition_for_list_of_dicts)
print('Filtered list of dictionaries:', filtered_list_of_dicts)

Filtered list of dictionaries: [{'name': 'Charlie', 'age': 35}]


we can also merge different data structure easily as a new nested one

In [21]:
# Usage Example 1: Merging dictionaries with overwriting
dicts_to_merge = [
    {'a': 1, 'b': 2},
    {'b': 3, 'c': 4}
]

merged_dict = li.nmerge(dicts_to_merge, dict_update=True)
print('Merged dictionaries with overwriting:', merged_dict)

Merged dictionaries with overwriting: {'a': 1, 'b': 3, 'c': 4}


In [22]:
# Usage Example 2: Merging dictionaries without overwriting
dicts_to_merge = [
    {'a': 1, 'b': 2},
    {'b': 3, 'c': 4}
]

# creating unique keys for duplicates
merged_dict = li.nmerge(dicts_to_merge, dict_update=False, dict_sequence=True, sequence_separator='_')
print('Merged dictionaries without overwriting:', merged_dict)

Merged dictionaries without overwriting: {'a': 1, 'b': 2, 'b_1': 3, 'c': 4}


In [23]:
# Usage Example 3: Merging lists with sorting
lists_to_merge = [
    [3, 1],
    [4, 2]
]

# Merge lists and sort the result
merged_list = li.nmerge(lists_to_merge, sort_list=True)
print('Merged and sorted lists:', merged_list)

Merged and sorted lists: [1, 2, 3, 4]
